# Final Project - Stock Price Prediction

## Table of Contents
1. [Purpose](#purpose)
2. [Read in libraries](#Read-in-libraries)
3. [Read in files](#read-in-files)
4. [make changes]

## Purpose 

* The purpose of this project is to develop a predictive model for forecasting the price of Halliburton Company (HAL)'s stock. Halliburton Company is an American multinational corporation and the world's second largest oil service company which is responsible for most of the world's largest fracking operations. The chosen approach involves utilizing the Random Forest algorithm along with additional predictors such as moving averages, RSI (Relative Strength Index), Bollinger Bands, and economic metrics such as interest rate, unemployment rate and crude oil price. By incorporating these predictors into the Random Forest model, the aim is to create a robust framework capable of generating accurate forecasts for HAL's stock prices.


## Method
* Predictors that will be used are:
    * simple moving average
    * exponential moving average
    * RSI (Relative strenght index)
    * Bollinger Bands
    * Interest rate
    * Unemployment rate
    * Crude Oil price
* Target variable
    * Closing price

## Read in libraries

In [2]:
import pandas as pd
import numpy as np 
import os

## Read in files
* The file was downloaded from Kaggle: 
    * https://www.kaggle.com/datasets/borismarjanovic/price-volume-data-for-all-us-stocks-etfs/data
        * The data contains full historical daily price and volume data for all US-based stocks and ETFs trading on the NYSE, NASDAQ, and NYSE MKT.
    * https://www.kaggle.com/datasets/varpit94/us-unemployment-rate-data 
        * Unemployment rate in US
    * https://www.kaggle.com/datasets/dennisholeski/historical-fed-funds
        * Interest rate
    * https://www.kaggle.com/datasets/mabusalah/brent-oil-prices
        * Crude oil price 


In [8]:
# reading in Halliburton 
halliburton = pd.read_csv('archive/Data/Stocks/hal.us.txt')
halliburton.head()

,Date,Open,High,Low,Close,Volume,OpenInt
0,1981-12-31,6.3200,6.3564,6.2739,6.2739,291277,0
1,1982-01-04,6.2739,6.3564,6.2009,6.3200,289093,0
2,1982-01-05,6.1551,6.2926,6.0087,6.0545,484812,0
3,1982-01-06,5.9354,5.9908,5.8534,5.9814,786150,0
4,1982-01-07,5.9354,5.9908,5.8534,5.9354,369567,0


In [9]:
# read in unemployment rate
unemployment = pd.read_csv('archive/Data/unemployment_rate/US UR.csv')
unemployment.head()

,Yearmon,US UnemploymentRate
0,01-01-1948,3.4
1,01-02-1948,3.8
2,01-03-1948,4.0
3,01-04-1948,3.9
4,01-05-1948,3.5


In [10]:
# read in interest rate
fed_rate = pd.read_csv('archive/Data/fed_rate/fed-funds-rate-historical-chart.csv')
fed_rate.head()

,date,value
0,7/1/1954,1.13
1,7/2/1954,1.25
2,7/3/1954,1.25
3,7/4/1954,1.25
4,7/5/1954,0.88


In [11]:
# read in oil price
oil_price = pd.read_csv('archive/Data/oil_price/BrentOilPrices.csv')
oil_price.head()

,Date,Price
0,20-May-87,18.63
1,21-May-87,18.45
2,22-May-87,18.55
3,25-May-87,18.60
4,26-May-87,18.63
